In [48]:
import sqlalchemy
import os
import pandas as pd
import pyodbc as py
import warnings
warnings.filterwarnings('ignore')

In [49]:
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=UG-BLR-L262;'
    r'database =AdventureWorksDW2008R2;'
    r'username=ETLdemo;'
    r'password=Devraj@@1;'
    r'Trusted_Connection=yes;'
)

In [50]:
cnxn = pyodbc.connect(conn_str)

In [51]:
def extract():
    try:
        src_conn=py.connect(conn_str)
        src_cursor=src_conn.cursor()
        sql="""Select table_name from AdventureWorksDW2008R2.INFORMATION_SCHEMA.TABLES 
            where TABLE_NAME in ('DimAccount','DimCurrency','DimCustomer','DimDate','DimDepartmentGroup')"""
        src_cursor.execute(sql)
        print(src_cursor)
        src_tables=src_cursor.fetchall()
        for tbl in src_tables:
            subquery="SELECT * FROM AdventureWorksDW2008R2.dbo.{}".format(tbl[0])
            df=pd.read_sql_query(subquery,src_conn)
            load(df, tbl[0])
    except exception as e:
              print("data extract error" + str(e))
    finally:
        src_conn.close()

In [52]:
#load data to ssms
def load(df,tbl):
    try:
        rows_imported=0
        server = 'UG-BLR-L262'
        database = 'ETLtest'
        sqlcon = sqlalchemy.create_engine('mssql+pyodbc://@' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
        sqlcon.connect()
        print("engine created")
        print("importing rows for {}".format(tbl))
        df.to_sql(tbl,schema='dbo',con=sqlcon, if_exists = 'replace', index=False)
        print("data loaded sucessfully")
    except exception as e:
        print("data load error "+ str(e))

In [53]:
try:
    extract()
except exception as e:
    print("error while loading data" + str(e))

engine created
importing rows for DimAccount
data loaded sucessfully
engine created
importing rows for DimCurrency
data loaded sucessfully
engine created
importing rows for DimCustomer
data loaded sucessfully
engine created
importing rows for DimDate
data loaded sucessfully
engine created
importing rows for DimDepartmentGroup
data loaded sucessfully
